# 欠損値とは何かを理解し，対応方法を知る(2022/03/17)

## 概要
---

本notebookでは，データ分析の前段階で発生する欠損データ・欠損値について調査し，それらへの対処方法を知ることを目的とする．データの欠損に対する基本的な立場は2つあり「なぜデータが欠損したか」という視点と，「欠損したデータをどう扱うか」という視点である．

「なぜデータが欠損したのか」という問いに対する答えは個々の事例によって変わってくるので，今回主として取り上げるのは「欠損したデータをどう扱うか」という点になる．ただし後者において選択する対処方法もデータ欠損の原因と密接に関わっていることから，本notebookで調査できる範囲も一般論的な薄いものになる．

欠損データ(欠測データ)処理は，当該領域だけでも専門書が一冊書ける深長なものらしい．筆者も適当なものを一冊読む予定だ．

## 欠損値(missing value)
---

欠損値(missing value)とは，欠測データ，欠落データとも呼ばれ，本来格納されるはずの変数に値が入っていないことを指す．実装ではNaNであったり，nullとして表現されることが多いだろう．

事象を観測することで得られるはずのデータが得られていない，ということであるから，欠損値に対して適切な対処をしないと分析結果に影響を及ぼす．欠損値の取り扱いについては様々な方法があるが，基本は「何かの値で埋める」か「当該データ・属性を削除(無視)する」か「そもそも欠損値の影響を受けづらい頑健な分析手法を用いる」かのいずれかに大別される．

欠損値が発生する原因は様々ある．例えば，アンケート調査における無回答や，調査データの転記忘れ，調査参加者の脱落などが挙げられる．

欠損値はその発生パターンに応じて，以下の3種類に分類できる．

### 完全にランダムな欠測(Missing Completely At Random; MCAR)

MCARはその名の通り，欠損値の発生タイミングが完全にランダムであることを指す．この場合は標本を十分な数で無作為抽出すれば，推論・推定が歪められることはない．しかし現実問題として，特定の原因に依存せず独立して欠損値が発生する事例というは極めてまれである．

### ランダムな欠測(Missing At Random; MAR)

MARは欠損値の発生タイミングが完全にランダムというわけではないが，欠損していない他の変数を用いてデータの欠落を完全に説明できる場合を指す．

現実の事象では欠損原因を完全に説明することは難しいので，原因だと考えられる属性で層別し，欠測発生のタイミングがランダムに近づかないかを考えることもあるらしい．

### ランダムではない欠測(Missing Not At Random; MNAR)

MNARはMCARやMARではないデータの欠損のことである．データ欠落の発生がランダムではなく，何かしらの原因が考えられることから「無視できない無回答」と呼ばれる．

MCARやMARと比較して，MNARを適切に処理しないと誤った結論を導く可能性が高い．

MNARの印象としては，原因だと思われる変数(属性)で層別し，MARとみなせないか考えるのが大切なような気がする．例えば，男女間でデータの欠測率に差があると考えられる場合，男性と女性の回答データを分けて考えることで，それぞれの集合ではデータの欠測はMARだとみなせる．

## 欠損値への対処方法
---

本節では，欠損値への対処方法として「当該データ・属性を削除(無視)する」方法と「何かの値で埋める」方法の代表的な手法を説明する．

### リストワイズ削除・完全ケース削除
この手法は欠損値を含む標本を分析前にすべて削除する手法である．MCARでない限りは標本データの偏りは必至なので，分析結果への影響を注意する必要がある．

ただし，処理が非常に単純であることから，後に紹介するペアワイズ削除よりも広く用いられているらしい．pandasではdropnaメソッドとして実装されている．

In [1]:
# リストワイズ削除
import pandas as pd
import numpy as np

series = pd.Series([1.2, 3.4, 5.6, np.nan, 7.8])
print(series)

series.dropna(inplace=True)
print(series)

0    1.2
1    3.4
2    5.6
3    NaN
4    7.8
dtype: float64
0    1.2
1    3.4
2    5.6
4    7.8
dtype: float64


### ペアワイズ削除

ペアワイズ削除はリストワイズ削除とは異なり，分析に使わない属性(変数)が欠落していた場合は，そのデータは削除せずに残す．分析によって用いる属性の組み合わせが異なれば，削除される標本数も異なる．

以下のDataFrameを用いた例では，生年月日の属性bornを分析に使わない属性だと仮定し，欠損値があってもデータを残している．分析とは関係のない変数(属性)の欠損値によってデータが削除されるのをできるだけ防ぐ意図が見て取れる．

In [2]:
# ペアワイズ削除
df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [np.nan, 'Batmobile', 'Bullwhip'],
                   "born": [pd.NaT, pd.Timestamp("1940-04-25"),
                            pd.NaT]})
df

,name,toy,born
0,Alfred,NaN,NaT
1,Batman,Batmobile,1940-04-25
2,Catwoman,Bullwhip,NaT


In [3]:
# 分析に必要な属性をサブセットとして指定する
df.dropna(subset=['name', 'toy'], inplace=True)
df

,name,toy,born
1,Batman,Batmobile,1940-04-25
2,Catwoman,Bullwhip,NaT


### ホットデッキ代入法

ホットデッキ代入法は欠損値を埋める代入法の一種であり，他の類似した標本から値をそのまま代入するという手法である．

ホットデッキ代入法の一種であるLOCF(Last Observation Carried Forward)では，ソートしたデータにおいて欠損値の直前の値を代入する．例えば時系列データであれば，微小時間前後で観測される値に大きな差はないと考えられるため，有効かもしれない．ただしLOCFはバイアス増大の懸念があるため推奨されていないらしい．

ホットデッキ代入法の発展として，コールドデッキ代入法という手法がある．こちらはバイアスの増大を抑えるために，同一のデータセットではなく類似した過去の調査結果から値を代入するという手法である．

pandasではfillnaメソッドを使うことでLOCFを再現できる．

In [4]:
# 欠損値を持つ適当なDataFrameを作成
np.random.seed(0)
df = pd.DataFrame(np.random.randn(5,5))
df.iloc[1,3] = np.nan
df.iloc[2,4] = np.nan
df.iloc[3,1] = np.nan
df

,0,1,2,3,4
0,1.764052,0.400157,0.978738,2.240893,1.867558
1,-0.977278,0.950088,-0.151357,NaN,0.410599
2,0.144044,1.454274,0.761038,0.121675,NaN
3,0.333674,NaN,-0.205158,0.313068,-0.854096
4,-2.552990,0.653619,0.864436,-0.742165,2.269755


In [5]:
# LOCF
df.fillna(method='ffill', inplace=True)
df

,0,1,2,3,4
0,1.764052,0.400157,0.978738,2.240893,1.867558
1,-0.977278,0.950088,-0.151357,2.240893,0.410599
2,0.144044,1.454274,0.761038,0.121675,0.410599
3,0.333674,1.454274,-0.205158,0.313068,-0.854096
4,-2.552990,0.653619,0.864436,-0.742165,2.269755


### 平均値代入法

平均値代入法では，その名の通り標本平均を欠損値に代入することで補完する．標本平均が変わらないというメリットがある一方，単一の値で埋めることになるので相関の低下が起こる．

平均値代入法も，pandasのfillnaメソッドを用いて実装できる．

In [6]:
# 欠損値を持つ適当なDataFrameを作成
np.random.seed(0)
df = pd.DataFrame(np.random.randn(5,5))
df.iloc[1,3] = np.nan
df.iloc[2,4] = np.nan
df.iloc[3,1] = np.nan
df

,0,1,2,3,4
0,1.764052,0.400157,0.978738,2.240893,1.867558
1,-0.977278,0.950088,-0.151357,NaN,0.410599
2,0.144044,1.454274,0.761038,0.121675,NaN
3,0.333674,NaN,-0.205158,0.313068,-0.854096
4,-2.552990,0.653619,0.864436,-0.742165,2.269755


In [7]:
df.mean()

0   -0.257699
1    0.864534
2    0.449539
3    0.483368
4    0.923454
dtype: float64

In [8]:
# 平均値代入法
df.fillna(df.mean(), inplace=True)
df

,0,1,2,3,4
0,1.764052,0.400157,0.978738,2.240893,1.867558
1,-0.977278,0.950088,-0.151357,0.483368,0.410599
2,0.144044,1.454274,0.761038,0.121675,0.923454
3,0.333674,0.864534,-0.205158,0.313068,-0.854096
4,-2.552990,0.653619,0.864436,-0.742165,2.269755


In [9]:
# 標本平均が変わっていないことを確認
df.mean()

0   -0.257699
1    0.864534
2    0.449539
3    0.483368
4    0.923454
dtype: float64

以上のようなシンプルな方法の他にも，回帰モデルを応用した手法など，バイアスを抑えるために様々な代入法が提案されている．

## 参考文献
---

[欠測データ，Wikipedia](https://ja.wikipedia.org/wiki/%E6%AC%A0%E6%B8%AC%E3%83%87%E3%83%BC%E3%82%BF)

[代入法(統計学)，Wikipedia](https://ja.wikipedia.org/wiki/%E4%BB%A3%E5%85%A5%E6%B3%95_(%E7%B5%B1%E8%A8%88%E5%AD%A6))

[欠損値とは？Pythonを使って欠損値の処理方法と実装を徹底解説【機械学習 入門編】，codExa](https://www.codexa.net/missing_value_python/)

[欠損データの処理，Practical Data Science with R and Python](https://uribo.github.io/practical-ds/03/handling-missing-data.html)

[欠損値の対処法，統計WEB](https://bellcurve.jp/statistics/blog/14238.html)

[欠損値の処理，PyQドキュメント](https://docs.pyq.jp/python/pydata/nan.html)